In [54]:
!pip install pyspark
from pyspark.sql.functions import when, col, filter
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("cdc").getOrCreate()

In [42]:
fldf = spark.read.csv("/content/LOAD00000001.csv")
fldf = fldf.withColumnRenamed("_c0","id").withColumnRenamed("_c1","FullName").withColumnRenamed("_c2", "City")
fldf.write.mode("overwrite").csv("/content/finalFile.csv", header= True)

In [58]:
udf = spark.read.csv("/content/20240803-012110611.csv")
udf = udf.withColumnRenamed("_c0", "action").withColumnRenamed("_c1","id").withColumnRenamed("_c2","FullName").withColumnRenamed("_c3", "City")
ffdf = spark.read.csv("/content/finalFile.csv", header=True)



In [59]:
for row in udf.collect():
  if row['action'] == 'U':
    ffdf = ffdf.withColumn("FullName", when(ffdf['id'] == row['id'], row['FullName']).otherwise(ffdf['FullName']))
    ffdf = ffdf.withColumn("City", when(ffdf['id'] == row['id'], ffdf['City']).otherwise(ffdf['City']))

  if row['action'] == 'I':
    insertedRow = [(row['id'], row['FullName'], row['City'])]
    columns = ['id', 'FullName', 'City']
    newdf = spark.createDataFrame(insertedRow, columns)
    ffdf = ffdf.union(newdf)

  if row['action'] == 'D':
    ffdf = ffdf.filter(ffdf.id != row['id'])

Row(action='U', id='7', FullName='ABC XYZ', City='Phoenix')
Row(action='I', id='130', FullName='Alica Bing', City='New York')
Row(action='I', id='131', FullName='Malinda Bing', City='Detroit')
Row(action='I', id='132', FullName='Chandler Bing', City='Portland')
Row(action='U', id='8', FullName='ABC XYZ', City='Denver')
Row(action='D', id='10', FullName='Jack Hicks', City='Houston')


In [60]:
display(ffdf)

DataFrame[id: string, FullName: string, City: string]